In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
from wordcloud import WordCloud
from textblob import TextBlob
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
nltk.download('vader_lexicon')
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_csv('/kaggle/input/amazon/amazon_reviews.csv')
data.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [3]:
data.isna().any()

Unnamed: 0              False
reviewerName             True
overall                 False
reviewText               True
reviewTime              False
day_diff                False
helpful_yes             False
helpful_no              False
total_vote              False
score_pos_neg_diff      False
score_average_rating    False
wilson_lower_bound      False
dtype: bool

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4915 non-null   int64  
 1   reviewerName          4914 non-null   object 
 2   overall               4915 non-null   float64
 3   reviewText            4914 non-null   object 
 4   reviewTime            4915 non-null   object 
 5   day_diff              4915 non-null   int64  
 6   helpful_yes           4915 non-null   int64  
 7   helpful_no            4915 non-null   int64  
 8   total_vote            4915 non-null   int64  
 9   score_pos_neg_diff    4915 non-null   int64  
 10  score_average_rating  4915 non-null   float64
 11  wilson_lower_bound    4915 non-null   float64
dtypes: float64(3), int64(6), object(3)
memory usage: 460.9+ KB


In [5]:
data = data.dropna()

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4913 entries, 1 to 4914
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4913 non-null   int64  
 1   reviewerName          4913 non-null   object 
 2   overall               4913 non-null   float64
 3   reviewText            4913 non-null   object 
 4   reviewTime            4913 non-null   object 
 5   day_diff              4913 non-null   int64  
 6   helpful_yes           4913 non-null   int64  
 7   helpful_no            4913 non-null   int64  
 8   total_vote            4913 non-null   int64  
 9   score_pos_neg_diff    4913 non-null   int64  
 10  score_average_rating  4913 non-null   float64
 11  wilson_lower_bound    4913 non-null   float64
dtypes: float64(3), int64(6), object(3)
memory usage: 499.0+ KB


In [7]:
data.reviewText[2031]

'[[ UPDATE - 6/19/2014 ]]So my lovely wife bought me a Samsung Galaxy Tab 4 for Father\'s Day and I\'ve been loving it ever since.  Just as other with Samsung products, the Galaxy Tab 4 has the ability to add a microSD card to expand the memory on the device.  Since it\'s been over a year, I decided to do some more research to see if SanDisk offered anything new.  As of 6/19/2014, their product lineup for microSD cards from worst to best (performance-wise) are the as follows:SanDiskSanDisk UltraSanDisk Ultra PLUSSanDisk ExtremeSanDisk Extreme PLUSSanDisk Extreme PRONow, the difference between all of these cards are simply the speed in which you can read/write data to the card.  Yes, the published rating of most all these cards (except the SanDisk regular) are Class 10/UHS-I but that\'s just a rating... Actual real world performance does get better with each model, but with faster cards come more expensive prices.  Since Amazon doesn\'t carry the Ultra PLUS model of microSD card, I had 

In [8]:
def remove_special_characters(text, remove_digits=False, remove_emojis=False):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s#@]', ' ', text)
    if remove_digits:
        text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    if remove_emojis: 
        text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = text.lower()
    
    return text

data['reviewText'] = data['reviewText'].apply(remove_special_characters, remove_digits=True, remove_emojis=False)

In [9]:
data.reviewText[2031]

'update so my lovely wife bought me a samsung galaxy tab for father s day and i ve been loving it ever since just as other with samsung products the galaxy tab has the ability to add a microsd card to expand the memory on the device since it s been over a year i decided to do some more research to see if sandisk offered anything new as of their product lineup for microsd cards from worst to best performance wise are the as follows sandisksandisk ultrasandisk ultra plussandisk extremesandisk extreme plussandisk extreme pronow the difference between all of these cards are simply the speed in which you can read write data to the card yes the published rating of most all these cards except the sandisk regular are class uhs i but that s just a rating actual real world performance does get better with each model but with faster cards come more expensive prices since amazon doesn t carry the ultra plus model of microsd card i had to do direct comparisons between the sandisk ultra extreme and 

In [10]:
dup1 = data.copy()
dup2 = data.copy()

In [11]:
dup1.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
1,1,0mie,5.0,purchased this for my device it worked as adve...,2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected i should have sprung for ...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,this think has worked out great had a diff bra...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,bought it with retail packaging arrived legit ...,2013-07-13,513,0,0,0,0,0.0,0.0
5,5,2Cents!,5.0,it s mini storage it doesn t do anything else ...,2013-04-29,588,0,0,0,0,0.0,0.0


In [12]:
dup2.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
1,1,0mie,5.0,purchased this for my device it worked as adve...,2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected i should have sprung for ...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,this think has worked out great had a diff bra...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,bought it with retail packaging arrived legit ...,2013-07-13,513,0,0,0,0,0.0,0.0
5,5,2Cents!,5.0,it s mini storage it doesn t do anything else ...,2013-04-29,588,0,0,0,0,0.0,0.0


In [13]:
analyzer = SentimentIntensityAnalyzer()

for index, row in dup1['reviewText'].items():
    score = analyzer.polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']

    # Assign sentiment based on scores
    if neg > pos:
        dup1.loc[index, 'sentiment'] = "Negative"
    elif pos > neg:
        dup1.loc[index, 'sentiment'] = "Positive"
    else:
        dup1.loc[index, 'sentiment'] = "Neutral"

# Process `dup2` using TextBlob
dup2[['polarity', 'subjectivity']] = dup2['reviewText'].apply(
    lambda Text: pd.Series(TextBlob(Text).sentiment)
)

# Assign sentiment based on polarity from TextBlob
for index, row in dup2.iterrows():
    polarity = row['polarity']
    
    if polarity < 0:
        dup2.loc[index, 'sentiment'] = "Negative"
    elif polarity > 0:
        dup2.loc[index, 'sentiment'] = "Positive"
    else:
        dup2.loc[index, 'sentiment'] = "Neutral"


In [14]:
constraints = ['#581845','#C70039','#2E4053','#1ABC9C','#7F8C8D']
def categorical_variable_summary(df, column_name):
    fig = make_subplots(rows=1,cols=2,
                        subplot_titles=('Countplot','Percentages'),
                        specs=[[{"type": "xy"}, {'type':'domain'}]])

    fig.add_trace(go.Bar( y = df[column_name].value_counts().values.tolist(), 
                          x = [str(i) for i in df[column_name].value_counts().index], 
                          text = df[column_name].value_counts().values.tolist(),
                          textfont = dict(size=15),
                          name = column_name,
                          textposition = 'auto',
                          showlegend=False,
                          marker=dict(color = constraints,
                                      line=dict(color='#DBE6EC',
                                                width=1))),
                  row = 1, col = 1)
    
    fig.add_trace(go.Pie(labels= df[column_name].value_counts().keys(),
                         values= df[column_name].value_counts().values,
                         textfont = dict(size = 20),
                         textposition='auto',
                         showlegend = False,
                         name = column_name,
                         marker=dict(colors=constraints)),
                  row = 1, col = 2)
    
    fig.update_layout(title={'text': column_name,
                             'y':0.9,
                             'x':0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')
    
    iplot(fig)

In [15]:
categorical_variable_summary(dup1,'sentiment')

In [16]:
categorical_variable_summary(dup2,'sentiment')

In [17]:
dup1.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,sentiment
1,1,0mie,5.0,purchased this for my device it worked as adve...,2013-10-25,409,0,0,0,0,0.0,0.0,Negative
2,2,1K3,4.0,it works as expected i should have sprung for ...,2012-12-23,715,0,0,0,0,0.0,0.0,Negative
3,3,1m2,5.0,this think has worked out great had a diff bra...,2013-11-21,382,0,0,0,0,0.0,0.0,Positive
4,4,2&amp;1/2Men,5.0,bought it with retail packaging arrived legit ...,2013-07-13,513,0,0,0,0,0.0,0.0,Positive
5,5,2Cents!,5.0,it s mini storage it doesn t do anything else ...,2013-04-29,588,0,0,0,0,0.0,0.0,Neutral


In [18]:
merged = dup1.merge(dup2, on='reviewText', suffixes=('_dup1', '_dup2'))

# Filter rows where sentiment columns differ
diff_sentiment = merged[merged['sentiment_dup1'] != merged['sentiment_dup2']]
diff_sentiment

,Unnamed: 0_dup1,reviewerName_dup1,overall_dup1,reviewText,reviewTime_dup1,day_diff_dup1,helpful_yes_dup1,helpful_no_dup1,total_vote_dup1,score_pos_neg_diff_dup1,...,day_diff_dup2,helpful_yes_dup2,helpful_no_dup2,total_vote_dup2,score_pos_neg_diff_dup2,score_average_rating_dup2,wilson_lower_bound_dup2,polarity,subjectivity,sentiment_dup2
0,1,0mie,5.0,purchased this for my device it worked as adve...,2013-10-25,409,0,0,0,0,...,409,0,0,0,0,0.0,0.000000,0.200000,0.200000,Positive
1,2,1K3,4.0,it works as expected i should have sprung for ...,2012-12-23,715,0,0,0,0,...,715,0,0,0,0,0.0,0.000000,0.129167,0.525000,Positive
4,5,2Cents!,5.0,it s mini storage it doesn t do anything else ...,2013-04-29,588,0,0,0,0,...,588,0,0,0,0,0.0,0.000000,-0.025000,0.700000,Negative
7,8,4evryoung,5.0,works in a htc rezound was running short of sp...,2014-03-24,259,1,0,1,1,...,259,1,0,1,1,1.0,0.206549,0.100000,0.450000,Positive
10,11,98020,3.0,it works but file writes are a bit slower than...,2013-11-20,383,0,0,0,0,...,383,0,0,0,0,0.0,0.000000,-0.033333,0.175000,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,4902,Zero Hero,5.0,what not to like works as specified recognized...,2013-12-12,361,0,0,0,0,...,361,0,0,0,0,0.0,0.000000,0.300000,0.512500,Positive
4905,4903,Z Guy,4.0,i purchased the sandisk ultra gb microsdhc for...,2014-11-01,37,0,0,0,0,...,37,0,0,0,0,0.0,0.000000,0.250000,0.366667,Positive
4906,4904,zhengziying,1.0,do not put any important data in it do not use...,2012-05-12,940,1,1,2,0,...,940,1,1,2,0,0.5,0.094531,0.017771,0.431450,Positive
4908,4906,Zigcarruse,5.0,it worked just like you would expect a micro s...,2014-02-13,298,0,0,0,0,...,298,0,0,0,0,0.0,0.000000,0.000000,0.000000,Neutral


In [19]:
result = diff_sentiment[['reviewText', 'sentiment_dup1', 'sentiment_dup2']]
result

,reviewText,sentiment_dup1,sentiment_dup2
0,purchased this for my device it worked as adve...,Negative,Positive
1,it works as expected i should have sprung for ...,Negative,Positive
4,it s mini storage it doesn t do anything else ...,Neutral,Negative
7,works in a htc rezound was running short of sp...,Negative,Positive
10,it works but file writes are a bit slower than...,Neutral,Negative
...,...,...,...
4904,what not to like works as specified recognized...,Negative,Positive
4905,i purchased the sandisk ultra gb microsdhc for...,Negative,Positive
4906,do not put any important data in it do not use...,Negative,Positive
4908,it worked just like you would expect a micro s...,Positive,Neutral


In [20]:
result.reviewText[0]

'purchased this for my device it worked as advertised you can never have too much phone memory since i download a lot of stuff this was a no brainer for me'

In [21]:
result.reviewText[4909]

'microsd card works flawlessly with my digital video recorder no problems at all with playback more words needed for official review'

In [22]:
result.reviewText[4908]

'it worked just like you would expect a micro sd card to work no noticeable issues have come up yet'